In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D, Embedding
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
tf.set_random_seed(seed_val)

In [50]:
def conv_model_multi(vocab_size, n_seq, embed_size, n_consonant, n_vowels, n_units):
    root_word_input = Input(shape=(embed_size,), dtype='int32', name="root_word_input")
    
    x = Embedding(vocab_size, embed_size, input_length=n_seq)(root_word_input)
#     x = GRU(n_units)(x)
    
    consonant_decoder_gru = GRU(n_units, return_sequences=True, name="consonant_decoder_gru")
    consonant_decoder_outputs = consonant_decoder_gru(x)
    
    vowel_decoder_gru = GRU(n_units, return_sequences=True, name="vowl_decoder_gru")
    vowel_decoder_outputs = vowel_decoder_gru(x)

    consonant_decoder_dense = Dense(n_consonant, activation='softmax', name="consonant_output")
    consonant_decoder_outputs = consonant_decoder_dense(consonant_decoder_outputs)
    
    vowel_decoder_dense = Dense(n_vowels, activation='softmax', name="vowel_output")
    vowel_decoder_outputs = vowel_decoder_dense(vowel_decoder_outputs)
    
    main_model = Model(root_word_input, [consonant_decoder_outputs, vowel_decoder_outputs])
    
    
    return main_model

In [ ]:
def generate_word_images_multi(words, int_words, char2tup, batch_size, n_consonant, n_vowels, seq_length):
    """

    returns cbow input of integer sequence of words. the inputs are for RNN where the context, is normal
    but for the decoder, there is an input and output

    """
    targets, target_inputs = [], []
    for word in words:
        target = word  # + '|'
        target_input = '&' + target + '|'
        targets.append(target)
        target_inputs.append(target_input)
    batch = 0
    n_batchs = len(words) // batch_size
    n_chars = 13
    while True:
        batch_words = words[batch:batch + batch_size]
        batch_int_words = get_context_words(int_words)
        batch_inputs = np.ndarray((batch_size, 1), dtype=np.float32)
        
        batch_cons_output = np.ndarray(
            (batch_size, n_chars, n_consonant), dtype=np.float32)
        batch_vow_output = np.ndarray(
            (batch_size, n_chars,  n_vowels), dtype=np.float32)
        
        for i in range(batch_size):
            input_con, input_vow = word2vec_seperated(char2tup,
                                                      batch_words[i], n_chars, n_consonant, n_vowels)

            batch_inputs[i][0] = 

            batch_cons_output[i] = input_con
            batch_vow_output[i] = input_vow
        yield [batch_inputs, batch_cons_dec_inputs, batch_vow_dec_inputs], [batch_cons_output, batch_vow_output]
        batch += 1
        if batch == n_batchs:
            batch = 0


In [51]:
train = conv_model_multi(20000, 6, 128, 40, 10, 256)

In [52]:
train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
root_word_input (InputLayer)    (None, 128)          0                                            
__________________________________________________________________________________________________
embedding_19 (Embedding)        (None, 6, 128)       2560000     root_word_input[0][0]            
__________________________________________________________________________________________________
consonant_decoder_gru (GRU)     (None, 6, 256)       295680      embedding_19[0][0]               
__________________________________________________________________________________________________
vowl_decoder_gru (GRU)          (None, 6, 256)       295680      embedding_19[0][0]               
__________________________________________________________________________________________________
consonant_